## Unsupervised ML on the Descartes Labs Platform: Interactive Deployment with Dynamic Compute
This notebook will demonstrate a typical example of how to deploy a ML model using Descartes Labs Platform APIs. General steps we will cover in this notebook are:
* Retrieving a running Batch Compute Function
* Displaying results overlain on input imagery in a web map with Dynamic Compute
* Specifying new AOIs to expand our model interactively using widgets

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import properties as p
import descarteslabs.dynamic_compute as dc

from descarteslabs.compute import Function, FunctionStatus, Job
from descarteslabs.dynamic_compute import ImageStack, Mosaic

In [ ]:
import geopandas as gpd
from datetime import datetime
from ipyleaflet import DrawControl

In [ ]:
org = dl.auth.Auth().payload["org"]
user_id = dl.auth.Auth().namespace

If you lost your ID, check it in [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) or search the latest created Function with the name as below:

In [ ]:
func_search = (
    Function.search()
    .filter(p.owner == user_id)
    .filter(p.name.startswith("Run KMeans Model Inference"))
    .sort(-Function.creation_date)
    .limit(1)
).collect()

for func in func_search:
    print(func.id)
    print(func.creation_date)

In [ ]:
async_func = func_search[0]
async_func

Here we'll create a Sentinel-2 stack of our time period:

In [ ]:
s2_stack = ImageStack.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "nir red green",
    start_datetime="2023-06-01",
    end_datetime="2023-09-01",
).filter(lambda x: x.cloud_fraction < 0.1)

Declaring a Dynamic Compute ipyleaflet map:

In [ ]:
m = dc.map
m.center = 44.4729, -73.1657
m.zoom = 13

And a simple DrawControl widget:

In [ ]:
# This is some interactivity with the map we'll embed below:
draw_control = DrawControl()

draw_control.polygon = {
    "shapeOptions": {"fillColor": "green", "color": "blue", "fillOpacity": 0.5},
    "drawError": {"color": "red", "message": "Oops!"},
    "allowIntersection": False,
}

# Add this to the same cell as the Draw Control handler function
feature_collection = {"type": "FeatureCollection", "features": []}

# Define this handle_draw function for the Draw Control widget
def handle_draw(target, action, geo_json):
    # Clears feature collection on each new polygon with new geojson
    feature_collection["features"] = [geo_json]


# Adding the handle_draw function to the Draw Control widget
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

Finally visualize our Dynamic Compute layers on the map:

In [ ]:
s2_stack.mean(axis="images").visualize("Sentinel-2", m)

*Note*

Our result's Mosaic object will "refresh" it's tile layers upon re-instantiation of it's class, as in the cell below. If you are waiting for your Batch Compute Function to process in real time you would need to re-run the following cell to update your imagery:

In [ ]:
kmeans_pid = f"{org}:kmeans-results-{user_id}"
kmeans_mosaic = Mosaic.from_product_bands(kmeans_pid, "class")
kmeans_mosaic.visualize("KMeans Results", m)

In [ ]:
m

In [ ]:
# dir(m)

Example of extracting a drawn polygon into a new list of arguments for our running Function:

In [ ]:
drawn_gdf = gpd.GeoDataFrame.from_features(feature_collection, crs=4326)
# Create a new set of DLTiles for this new AOI
geocontext_geom = drawn_gdf["geometry"][0]
# You could also pass the map's geocontext as a WKT
# geocontext_geom = box(*wf.map.geocontext().bounds)
dltiles = dl.geo.DLTile.from_shape(
    geocontext_geom, resolution=10.0, tilesize=1024, pad=0
)
args = [(dltile.key, kmeans_pid) for dltile in dltiles]
len(args)

In [ ]:
jobs = async_func.map(args)
len(jobs)